In [2]:
import csv
import os
from matplotlib import pyplot as plt
import cv2
import numpy as np
import random
from collections import namedtuple

USE_CV2_IMREAD = True
SIDE_CAMERA_OFFSET = 0.3


class DataLoader:
    
    FrameRecord = namedtuple('FrameRecord', ['center_path', 'left_path', 'right_path', 'angle', 'dup_factor'])
    
    def __init__(
            self, data_dirs_and_dup_factors,
            batch_size=32):
        self.data_dirs_and_dup_factors = data_dirs_and_dup_factors
        self.batch_size = batch_size
        
        self.frame_records = []
        for d, dup_factor in self.data_dirs_and_dup_factors:
            with open(os.path.join(d, 'driving_log.csv')) as inf:
                for line in csv.reader(inf):
                    paths = line[:3]
                    paths = [os.path.join(d, 'IMG', p.split('/')[-1]) for p in paths]
                    angle = float(line[3])
                    self.frame_records.append(self.FrameRecord(*(paths + [angle, dup_factor])))

    @staticmethod
    def imread(path):
        if USE_CV2_IMREAD:
            imdata = cv2.imread(path)
        else:
            imdata = plt.imread(path)
            imdata = np.cast(np.round(imdata * 255), np.uint8)
        
        return imdata

    def load_data(self):
        raise Exception('bitrotted')
        batch_x = []
        batch_y = []
        for line in self.csv_lines:
            forward_img_path = self.fix_img_path(line[0])
            imdata = self.imread(forward_img_path)
            
            batch_x.append(imdata)
            batch_y.append(float(line[3]))
            
            if len(batch_x) == BATCH_SIZE:
                yield np.array(batch_x), np.array(batch_y)
                batch_x, batch_y = [], []
        
    def num_batches(self):
        return len(self.csv_lines) // BATCH_SIZE
        
    @staticmethod
    def fix_img_path(path):
        fname = path.split('/')[-1]
        return os.path.join(os.getcwd(), 'data', 'IMG', fname)
    
    @staticmethod
    def all_img_paths(path):
        center = fix_img_path(path)
        left = 'left'.join(center.rsplit('center', 1))
        right = 'right'.join(center.rsplit('right', 1))
        
        return center, left, right
    
    def load_all_data(self):
        rv_x = []
        rv_y = []
        for r in self.frame_records:
            offsets = [0, SIDE_CAMERA_OFFSET, -SIDE_CAMERA_OFFSET]
            for path, offset in zip(r[:3], offsets):
                imdata = self.imread(path)
                angle = r.angle + offset
                
                for i in range(r.dup_factor):
                    rv_x.append(imdata)
                    rv_y.append(angle)
                    # also append reversed data
                    rv_x.append(imdata[:,::-1,:])
                    rv_y.append(-angle)
        
        return np.array(rv_x), np.array(rv_y)

# def load_data():
#     with open('data/driving_log.csv') as inf:
#         batch = []
#         reader = csv.reader(inf)
#         for line in reader:
#             forward_img_path = fix_img_path(line[0])
#             imdata = imread(forward_img_path) * 255
#             batch.append((
#                 imdata,
#                 float(line[3])))
            
#             if len(batch) == BATCH_SIZE:
#                 yield batch
#                 batch = []

# forward_cam_data_x = np.array(forward_cam_data_x)
# forward_cam_data_y = np.array(forward_cam_data_y)

# print('{} samples'.format(len(forward_cam_data_x)))

In [3]:
BATCH_SIZE = 32

data_loader = DataLoader(
    [('data_driving_straight', 1), ('data_recovery', 2), ('data_jungle_track', 0), ('data_recovery2', 2)],
    batch_size=BATCH_SIZE)

In [4]:
from keras import models, layers

EPOCHS = 10
USE_GENERATOR_LOADER = False
USE_DEEP_MODEL = True
USE_DEEPER_MODEL = True


model = models.Sequential()
model.add(layers.Cropping2D(cropping=((70, 25), (0, 0)), input_shape=(160, 320, 3)))
model.add(layers.Lambda(lambda x: (x - 128.0) / 2.0))

if USE_DEEPER_MODEL:
    model.add(layers.Conv2D(24, (5, 5), strides=(2, 2), activation='elu'))
    model.add(layers.Conv2D(36, (5, 5), strides=(2, 2), activation='elu'))
    model.add(layers.Conv2D(48, (5, 5), strides=(2, 2), activation='elu'))
    model.add(layers.Conv2D(64, (3, 3), activation='elu'))
    model.add(layers.Conv2D(64, (3, 3), activation='elu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(1164, activation='elu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(100, activation='elu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(50, activation='elu'))
    model.add(layers.Dense(10, activation='elu'))
    model.add(layers.Dense(1))
elif USE_DEEP_MODEL:
    model.add(layers.Conv2D(6, (5, 5), activation='elu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(16, (5, 5), activation='elu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation='elu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(84))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1))
else:
    model.add(layers.Flatten())
    model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mse')

if USE_GENERATOR_LOADER:
    model.fit_generator(data_loader.load_data(),
                        data_loader.num_batches(),
                        epochs=EPOCHS)
else:
    x, y = data_loader.load_all_data()
    model.fit(x, y, validation_split=0.2, epochs=EPOCHS, shuffle=True)

model.save('model1.h5')

Train on 64401 samples, validate on 16101 samples
Epoch 1/10
64401/64401 [==============================] - 68s 1ms/step - loss: 236.1564 - val_loss: 0.1635
Epoch 2/10
64401/64401 [==============================] - 65s 1ms/step - loss: 0.1430 - val_loss: 0.1612
Epoch 3/10
64401/64401 [==============================] - 65s 1ms/step - loss: 0.1423 - val_loss: 0.1611
Epoch 4/10
64401/64401 [==============================] - 65s 1ms/step - loss: 617.3271 - val_loss: 0.1619
Epoch 5/10
64401/64401 [==============================] - 65s 1ms/step - loss: 0.1425 - val_loss: 0.1611
Epoch 6/10
64401/64401 [==============================] - 65s 1ms/step - loss: 0.1423 - val_loss: 0.1611
Epoch 7/10
64401/64401 [==============================] - 64s 1ms/step - loss: 0.1433 - val_loss: 0.1611
Epoch 8/10
64401/64401 [==============================] - 65s 1ms/step - loss: 0.4447 - val_loss: 0.1611
Epoch 9/10
64401/64401 [==============================] - 65s 1ms/step - loss: 967.3526 - val_loss: 0.1653

In [28]:
model.save('model1.h5')

In [5]:
x, y = data_loader.load_all_data()

In [6]:
%matplotlib inline
from matplotlib import pyplot as plt

#plt.imshow(x[0] / 255)